In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Summary Statistics

In [ ]:
from collections import namedtuple
Record = namedtuple('Record',['desc','value1','value2'])

In [ ]:
recDF = sc.parallelize([Record("first",1,3.7),
                        Record("second",-2,2.1),
                        Record("third",6,0.7)]).toDF()

In [ ]:
recStats = recDF.describe()
recStats.show()

In [ ]:
recStatsPandas = recStats.toPandas().set_index('summary')
recStatsPandas

In [ ]:
recStatsPandas.loc['mean'].value1

In [ ]:
recDF.stat.corr('value1','value2')

In [ ]:
recDF.stat.cov('value1','value2')

In [ ]:
recDF.stat.freqItems(['value1','value2']).toPandas()

## Sampling

In [ ]:
df = sqlc.createDataFrame([(1, 10), (1, 20), (2, 10),(2, 20), (2, 30), (3, 20), (3, 30)]).toDF("key", "value")

In [ ]:
df.show()

In [ ]:
dfSampled = df.sample(withReplacement=False, fraction=0.3, seed=11)
dfSampled.show()

In [ ]:
training, testing = df.randomSplit(weights=[0.3, 0.7], seed=11)

In [ ]:
training.show()

In [ ]:
testing.show()

## Stratified Sampling

In [ ]:
dfStrat = df.stat.sampleBy(col="key", fractions={1: 0.7, 2: 0.7, 3: 0.7}, seed=11)
dfStrat.show()

## Random Data Generation

In [ ]:
from pyspark.sql.functions import rand, randn

In [ ]:
df = sqlc.range(0, 5)

In [ ]:
df.show()

In [ ]:
df2 = df.select("id").withColumn("uniform", rand(5)).withColumn("normal", randn(5))

In [ ]:
df2.show()

In [ ]:
sc.stop()